In [1]:
import os, sys
from os.path import expanduser
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

user_home = expanduser("~")
neuron_home = '%s/bin/nrnpython3/lib/python' % user_home
model_home = '%s/src/model' % user_home
dentate_home = '%s/dentate' % model_home
datasets_home = '/media/igr/d865f900-7fcd-45c7-a7a7-bd2a7391bc40/Data/DG/'
results_prefix = os.getenv('DG_RESULTS_DIR', '%s/results' % datasets_home)
sys.path.append(neuron_home) 
sys.path.append(model_home) 

import neuroh5
from neuroh5.io import read_population_ranges, read_population_names
import dentate
from dentate import utils, plot, stimulus, spikedata
from dentate.env import Env
from dentate.utils import Struct, zip_longest

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module '_yaml' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
bin_size = 5.
config_path = '%s/config/Full_Scale_Basis.yaml' % dentate_home
template_path = '%s/templates' % dentate_home

rel_spike_events_paths = ['Full_Scale_GC_Exc_Sat_DD_SLN_Diag_10618023.bw/dentatenet_Full_Scale_GC_Exc_Sat_DD_SLN_results_compressed.h5', 
                          'Full_Scale_GC_Exc_Sat_DD_SLN_Diag_remap_10619524.bw/dentatenet_Full_Scale_GC_Exc_Sat_DD_SLN_results_compressed.h5'
                          ]

trajectory_path = '%s/Full_Scale_Control/DG_remap_spike_trains_20191113_compressed.h5' % datasets_home

spike_input_paths = [ '%s/%s' % (results_prefix, spike_events_path) for spike_events_path in rel_spike_events_paths ]
spike_namespace_id = 'Spike Events'

arena_ids = ['A', 'Remap A']
trajectory_id = 'Diag'


In [13]:
        
baks_config = Struct(**{'BAKS Alpha': 4.77,'BAKS Beta': None})
pf_config = Struct(**{'Minimum Width': 15., 'Minimum Rate': 1.0})

env = None
if config_path is not None:
    env = Env(config_file=config_path, template_paths=template_path)
    if env.analysis_config is not None:
        baks_config.update(env.analysis_config['Firing Rate Inference'])
        pf_config.update(env.analysis_config['Place Fields'])


n_inputs = len(spike_input_paths)
assert(n_inputs > 1)
            
spike_namespace_ids = [spike_namespace_id]*n_inputs

assert(len(spike_namespace_ids) == n_inputs)
assert(len(arena_ids) == n_inputs)
            
(population_ranges, N) = read_population_ranges(spike_input_paths[0])
population_names = read_population_names(spike_input_paths[0])

pop_num_cells = {}
for k in population_names:
    pop_num_cells[k] = population_ranges[k][1]

populations = list(population_names)


trajectory = stimulus.read_trajectory(trajectory_path, arena_ids[0], trajectory_id)

In [14]:
print(populations)
print(trajectory)

['AAC', 'BC', 'CA3c', 'GC', 'HC', 'HCC', 'IS', 'LPP', 'MC', 'MOPP', 'MPP', 'NGFC']
(array([-107.5     , -107.47    , -107.44    , ...,   99.95566 ,
         99.97687 ,   99.998085], dtype=float32), array([-100.      , -100.      , -100.      , ...,   99.95566 ,
         99.97687 ,   99.998085], dtype=float32), array([ -7.5 ,  -7.47,  -7.44, ..., 282.78, 282.81, 282.84], dtype=float32), array([-250., -249., -248., ..., 9426., 9427., 9428.], dtype=float32))


In [33]:

populations=['GC', 'MC']
PF_arena_dicts = defaultdict(lambda: defaultdict(lambda: None))
for spike_input_path, spike_namespace_id, arena_id in zip_longest(spike_input_paths, spike_namespace_ids, arena_ids):
        
    this_spike_namespace = '%s %s %s' % (spike_namespace_id, arena_id, trajectory_id)
    this_spike_namespace = spike_namespace_id
    print('reading %s' % spike_input_path)
    spkdata = spikedata.read_spike_events(spike_input_path, populations, this_spike_namespace,
                                          spike_train_attr_name='t', time_range=[500., 9500.])

    spkpoplst = spkdata['spkpoplst']
    spkindlst = spkdata['spkindlst']
    spktlst = spkdata['spktlst']
    num_cell_spks = spkdata['num_cell_spks']
    pop_active_cells = spkdata['pop_active_cells']
    tmin = spkdata['tmin']
    tmax = spkdata['tmax']

    time_range = [tmin, tmax]
    time_bins  = np.arange(time_range[0], time_range[1], bin_size)

    for i, subset in enumerate(spkpoplst):

        spkts         = spktlst[i]
        spkinds       = spkindlst[i]
        spkdict       = spikedata.make_spike_dict(spkinds, spkts)

        rate_bin_dict = spikedata.spike_density_estimate(subset, spkdict, time_bins, arena_id=arena_id,
                                                             trajectory_id=trajectory_id, progress=True, **baks_config())
        PF_dict = spikedata.place_fields(subset,  65., rate_bin_dict, trajectory, arena_id=arena_id,
                                         trajectory_id=trajectory_id, progress=True, **pf_config())
        
        PF_arena_dicts[arena_id][subset] = PF_dict

 

reading /media/igr/d865f900-7fcd-45c7-a7a7-bd2a7391bc40/Data/DG//results/Full_Scale_GC_Exc_Sat_DD_SLN_Diag_10618023.bw/dentatenet_Full_Scale_GC_Exc_Sat_DD_SLN_results_compressed.h5


100%|██████████| 22865/22865 [02:32<00:00, 150.16it/s]


reading /media/igr/d865f900-7fcd-45c7-a7a7-bd2a7391bc40/Data/DG//results/Full_Scale_GC_Exc_Sat_DD_SLN_Diag_remap_10619524.bw/dentatenet_Full_Scale_GC_Exc_Sat_DD_SLN_results_compressed.h5


100%|██████████| 17825/17825 [01:58<00:00, 150.69it/s]


In [34]:
GC_cells_A = set([k for k in PF_arena_dicts['A']['GC'].keys() if PF_arena_dicts['A']['GC'][k]['pf_count'] > 0])
GC_cells_A_remap = set([k for k in PF_arena_dicts['Remap A']['GC'].keys() if PF_arena_dicts['Remap A']['GC'][k]['pf_count'] > 0])

print(len(GC_cells_A))
print(len(GC_cells_A_remap))

GC_A_remap_intersection = GC_cells_A.intersection(GC_cells_A_remap)
print(len(GC_A_remap_intersection))

5503
292
120


In [35]:
MC_cells_A = set([k for k in PF_arena_dicts['A']['MC'].keys() if PF_arena_dicts['A']['MC'][k]['pf_count'] > 0])
MC_cells_A_remap = set([k for k in PF_arena_dicts['Remap A']['MC'].keys() if PF_arena_dicts['Remap A']['MC'][k]['pf_count'] > 0])

print(len(MC_cells_A))
print(len(MC_cells_A_remap))

MC_A_remap_intersection = MC_cells_A.intersection(MC_cells_A_remap)
print(len(MC_A_remap_intersection))

15107
11914
9082


In [36]:
GC_first_gid_A = next(iter(GC_cells_A))
print(PF_arena_dicts['A']['GC'][GC_first_gid_A])

{'pf_count': array([1], dtype=uint32), 'pf_mean_width': array([40.949997], dtype=float32), 'pf_mean_rate': array([1.0186007], dtype=float32), 'pf_peak_rate': array([1.0686725], dtype=float32), 'pf_mean_norm_rate': array([0.7759876], dtype=float32), 'pf_x_locs': [29.880490243434906], 'pf_y_locs': [29.880490243434906]}


In [37]:
MC_first_gid_A = next(iter(MC_cells_A))

print(PF_arena_dicts['A']['MC'][MC_first_gid_A])

{'pf_count': array([1], dtype=uint32), 'pf_mean_width': array([31.200005], dtype=float32), 'pf_mean_rate': array([2.675638], dtype=float32), 'pf_peak_rate': array([3.09439], dtype=float32), 'pf_mean_norm_rate': array([2.0353608], dtype=float32), 'pf_x_locs': [-60.434722661972046], 'pf_y_locs': [-60.434722661972046]}


In [38]:
A_pf_dict = PF_arena_dicts['A']['GC']
A_remap_pf_dict = PF_arena_dicts['Remap A']['GC']
x_locs = []
remap_x_locs = []
for gid in GC_A_remap_intersection:
    
    A_pf_locs = A_pf_dict[gid]['pf_x_locs']
    A_remap_pf_locs = A_remap_pf_dict[gid]['pf_x_locs']
    if len(A_pf_locs) == len(A_remap_pf_locs):
        x_locs.append(A_pf_locs)
        remap_x_locs.append(A_remap_pf_locs)
    else:
        x_locs.append(np.asarray([A_pf_locs[0]]))
        remap_x_locs.append(np.asarray([A_remap_pf_locs[0]]))
    
x_locs_array = np.concatenate(x_locs)
remap_x_locs_array = np.concatenate(remap_x_locs)
print(len(x_locs_array))
print(len(remap_x_locs_array))
print(np.corrcoef(x_locs_array, remap_x_locs_array))


121
121
[[ 1.        -0.1855617]
 [-0.1855617  1.       ]]


In [40]:
A_pf_dict = PF_arena_dicts['A']['MC']
A_remap_pf_dict = PF_arena_dicts['Remap A']['MC']
x_locs = []
remap_x_locs = []
for gid in MC_A_remap_intersection:
    
    A_pf_locs = A_pf_dict[gid]['pf_x_locs']
    A_remap_pf_locs = A_remap_pf_dict[gid]['pf_x_locs']
    if len(A_pf_locs) == len(A_remap_pf_locs):
        x_locs.append(A_pf_locs)
        remap_x_locs.append(A_remap_pf_locs)
    else:
        x_locs.append(np.asarray([A_pf_locs[0]]))
        remap_x_locs.append(np.asarray([A_remap_pf_locs[0]]))
    
x_locs_array = np.concatenate(x_locs)
remap_x_locs_array = np.concatenate(remap_x_locs)
print(len(x_locs_array))
print(len(remap_x_locs_array))
print(np.corrcoef(x_locs_array, remap_x_locs_array))

9221
9221
[[1.         0.02238542]
 [0.02238542 1.        ]]
